# image scrape


In [ ]:
import spotipy
import spotipy.util as util
import os
import sys
import pprint
import string
import traceback
import urllib.request as req

In [ ]:
SPOTIPY_CLIENT_ID     = 'client-id'
SPOTIPY_CLIENT_SECRET = 'client-secret'
SPOTIPY_REDIRECT_URI  = 'redirect-url'
my_username = "my-username"
os.environ['SPOTIPY_CLIENT_ID']     = SPOTIPY_CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = SPOTIPY_CLIENT_SECRET
os.environ['SPOTIPY_REDIRECT_URI']  = SPOTIPY_REDIRECT_URI

# permissions
my_scope = 'user-library-read user-follow-read playlist-read-collaborative playlist-read-private playlist-modify-private playlist-modify-public'

token = util.prompt_for_user_token(
        username=my_username,
        scope=my_scope,
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET,
        redirect_uri=SPOTIPY_REDIRECT_URI)

In [ ]:
def get_all_playlists(spot):
    '''
    Get all the user's playlists
    
    Arguments:
        spot - spotipy object
        
    Returns:
        list - all user's playlists
    '''
    results = spot.current_user_playlists()
    all_playlists = results['items']
    while results['next']:
        results = spot.next(results)
        all_playlists.extend(results['items'])
    return all_playlists

def get_playlist_id(playlist, playlists):
    '''
    given a list of dictionaries of playlist attributes, check if user specified playlist exists
    
    Arguments:
        playlist - string, name of playlist to search for
        playlists - list of possible playlists
    
    Returns:
        int     - index in list of playlists where desired playlist found. -1 if not found 
    '''
    for i in range(0, len(playlists)):
        if playlists[i]['name'] == playlist:
            return playlists[i]['id']
    return None

def get_playlist_tracks(spot, user_id, playlist_id):
    '''
    '''
    results = spot.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = spot.next(results)
        tracks.extend(results['items'])
    return tracks

def get_artwork_url(track):
    '''
    '''
    # weed out local tracks
    if track['track']["is_local"] is not True:
        return track['track']['album']['images'][0]['url']
    return None

def prompt():
    '''
    '''
    return input("Which playlist would you like album art from?\n")

In [ ]:
if token: 
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    
    plist    = None
    plist_id = None
    user_id = (sp.me())['id']
    all_playlists = get_all_playlists(sp)
    
    while plist is None:
        plist = prompt()
        plist_id = get_playlist_id(plist, all_playlists)
    
    plist_tracks = get_playlist_tracks(sp, user_id, plist_id)
    
    destination_path = os.getcwd() + "/" + plist + "-artwork"
    if not os.path.exists(destination_path) or not os.path.isdir(destination_path):
        os.mkdir(destination_path)
        print("directory %s created" % destination_path)
else:
    print("fuck ur tkn")

In [ ]:
albums = set()
for track in plist_tracks:
    filename = track['track']['artists'][0]['name'] + "-" + track['track']['album']['name']
    try:
        if track is not None:
            url = get_artwork_url(track)
    except:
        print("Missing or no artwork for %s" % filename)
        pass
    albums.add((filename, url))
for album in albums:
    # forward slashes wonk up filenames
    album_and_song = album[0].replace("/", "~")
    file_path = destination_path + "/" + album_and_song + ".jpg"
    url = album[1]
    try:
        req.urlretrieve(url, file_path)
    except:
        print("Could not retrive artwork for '%s'" % album_and_song)
        pass
print("Done")